<a href="https://colab.research.google.com/github/RichardLin3/InfoSysProj/blob/main/ISProj_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

repo_url = "https://github.com/RichardLin3/InfoSysProj.git"
repo_dir = "InfoSysProj"

if os.path.exists(repo_dir) and os.listdir(repo_dir):
    print(f"Directory '{repo_dir}' already exists and is not empty. Skipping git clone.")
else:
    print(f"Cloning '{repo_url}' into '{repo_dir}'...")
    !git clone {repo_url}

Cloning 'https://github.com/RichardLin3/InfoSysProj.git' into 'InfoSysProj'...
Cloning into 'InfoSysProj'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 57 (delta 26), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 71.82 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
import json


with open('InfoSysProj/data/minors_v3.json', 'r') as f:
    minor_data = json.load(f)
with open('InfoSysProj/data/gened_v2.json', 'r') as f:
    gened_data = json.load(f)

with open('InfoSysProj/data/all-majors.json', 'r') as f:
    major_data = json.load(f)

with open('InfoSysProj/data/liberal_arts_courses.json', 'r') as f:
    lib_data = json.load(f)

with open('InfoSysProj/data/ID.json', 'r') as f:
    ID_data = json.load(f)

In [ ]:
major_names = list(major_data.keys())
minor_names = list(minor_data.keys())

In [ ]:
#major selection
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=major_names,
    description='Select a Major:',
    disabled=False,
)

def on_change(change):
    global selected_major
    if change['type'] == 'change' and change['name'] == 'value':
        selected_major = change['new']
        print(f"You selected: {selected_major}")

dropdown.observe(on_change)

display(dropdown)

Dropdown(description='Select a Major:', options=('Data Science', 'Applied Chemistry BS', 'Architecture Technol…

You selected: Communication Design
You selected: Data Science


In [ ]:
dropdown2 = widgets.Dropdown(
    options=minor_names,
    description='Select a Minor:',
    disabled=False,
)

def on_change2(change):
    global selected_minor
    if change['type'] == 'change' and change['name'] == 'value':
        selected_minor = change['new']
        print(f"You selected: {selected_minor}")

dropdown2.observe(on_change2)

display(dropdown2)

Dropdown(description='Select a Minor:', options=('Arabic Language and Cultural Studies', 'Art History & Visual…

You selected: Computer Science
You selected: Creative Writing


In [ ]:
#checks if a major/minor is selected otherwise default to the first
if selected_major is None:
  selected_major = major_names[0]
if minor_names is None:
  selected_minor = minor_names[0]

In [ ]:
courses = []  # Initialize an empty list to store course codes

if selected_minor in minor_data.keys():  # Check if the selected minor exists in minor_data
    minor_info = minor_data[selected_minor]
    for section_name, section_list in minor_info.items(): # Iterate through the items of the minor_info dictionary
        # section list is usually a list
        for entry in section_list:
            # Check if the entry is a dictionary and has a 'Course' key
            if isinstance(entry, dict) and 'Course' in entry:
                courses.append(entry['Course'])
            # Check if the entry is a dictionary and contains nested course options (like 'Choose 1')
            elif isinstance(entry, dict):
                for key, value in entry.items():
                    if isinstance(value, list):
                        for item in value:
                            if isinstance(item, dict) and 'Course' in item:
                                courses.append(item['Course'])


print(courses) # Print the extracted courses to verify

['ENG 1141', 'ENG 2142', 'ENG 2143', 'ENG 2144', 'ENG 2145', 'ENG 1151', 'ENG 2001', 'ENG 2002', 'ENG 2003', 'ENG 2420', 'ENG 2420ID', 'ENG 2720', 'ENG 3402', 'ENG 3403', 'ENG 3760']


^^ it checks if we found the right minor, then goes inside its “Requirements” and collects every course code.
if there’s a “choose 1” section, it digs into that too and adds those course codes to the list.

In [ ]:
gened_areas = ['EC', 'MQR', 'LPS', 'WCGI', 'USED', 'IS', 'CE', 'SW' ,'Lib Art', 'Adv Lib Art', 'ID', 'Major_Core','Major_Elec']
course_gened_fulfillment_list = []

In [ ]:
gened_fulfillment_map = {}
for core_type in ['Required Core', 'Flexible Core']:
    for course_info in gened_data['gened'][core_type]:
        areas = course_info.get('Area(s)', [])
        gened_fulfillment_map[course_info['Course']] = areas

In [ ]:
#set of course in the core and electives for followup processing
major_core = set([course['Course'] for course in major_data[selected_major]['Core']])
major_elective = set([course['Course'] for course in major_data[selected_major]['Electives']])
Adv_lib = set([course['Course'] for course in lib_data['Adv Lib Art']])
Lib_art = set([course['Course'] for course in lib_data['Lib Art']])

In [ ]:
import re

course_gened_fulfillment_list = [] # Clear the list before populating it

for course in courses:
    fulfilled_geneds = gened_fulfillment_map.get(course, [])

    if not isinstance(fulfilled_geneds, list):
            fulfilled_geneds = [fulfilled_geneds] if fulfilled_geneds else []

    # Check for major core and elective fulfillment
    if course in major_core:
        fulfilled_geneds.append('Major_Core')
    if course in major_elective:
        fulfilled_geneds.append('Major_Elec')

    # Check for Liberal Arts fulfillment
    if course in Lib_art:
        fulfilled_geneds.append('Lib Art')
    if course in Adv_lib:
        fulfilled_geneds.append('Lib Art')
        fulfilled_geneds.append('Adv Lib Art')

    # Check for ID fulfillment
    if course in ID_data or re.search(r'\d+ID$', course):
        fulfilled_geneds.append('ID')


    course_gened_fulfillment_list.append({
        'Courses in Minor': course,
        'Fulfilled Areas': fulfilled_geneds
    })

In [ ]:
import pandas as pd

course_fulfillment_list_for_df = []
gened_areas_for_columns = gened_areas

for course_info in course_gened_fulfillment_list:
    course_name = course_info['Courses in Minor']
    fulfilled_areas = course_info['Fulfilled Areas']
    course_dict = {'Courses in Minor': course_name}
    for gened_abbr in gened_areas_for_columns:
        course_dict[gened_abbr] = 'X' if gened_abbr in fulfilled_areas else ''
    course_fulfillment_list_for_df.append(course_dict)

course_fulfillment_df = pd.DataFrame(course_fulfillment_list_for_df)
course_fulfillment_df = course_fulfillment_df.fillna('')

# Set the index to start from 1
course_fulfillment_df.index = range(1, len(course_fulfillment_df) + 1)

# Display the DataFrame with dotted horizontal and vertical lines and dynamic caption
display(
    course_fulfillment_df.style
    .set_properties(**{'border': '1px dotted black'})
    .set_caption(f'{selected_major} Major and {selected_minor} Minor Course Fulfillment Overview')
    .set_table_styles([{'selector': 'caption', 'props': [('font-size', '20px')]}]
    )
)

,Courses in Minor,EC,MQR,LPS,WCGI,USED,IS,CE,SW,Lib Art,Adv Lib Art,ID,Major_Core,Major_Elec
1,ENG 1141,,,,,,,X,,X,,,,
2,ENG 2142,,,,,,,,,X,X,,,
3,ENG 2143,,,,,,,,,,,,,
4,ENG 2144,,,,,,,,,X,X,,,
5,ENG 2145,,,,,,,,,X,X,,,
6,ENG 1151,,,,,,X,,,X,,,,
7,ENG 2001,,,,,,,X,,X,X,,,
8,ENG 2002,,,,,,,X,,X,X,,,
9,ENG 2003,,,,,,,X,,X,X,,,
10,ENG 2420,,,,,,X,,,X,X,,,


In [ ]:
import pandas as pd
from typing import Dict, List, Any
# Assuming 'display' is available from IPython.display

# --- ASSUMED VARIABLES (Defined by your preceding code) ---
# ... (minor_data, gened_areas, course_gened_fulfillment_list, final_df)
# ------------------------------------------------------------------------

# --- 1. Processing Function (Unchanged for correctness) ---

def process_curriculum_json(curriculum_name: str, curriculum_data: Dict[str, Any],
                            gened_areas: List[str], fulfillment_list: List[Dict]) -> pd.DataFrame:
    """Processes the curriculum JSON into a MultiIndex DataFrame."""

    data = curriculum_data
    fulfillment_map = {item['Courses in Minor']: item['Fulfilled Areas'] for item in fulfillment_list}
    processed_list = []

    # --- CORE Courses ---
    core_section = data.get("Core", [])
    section_name = f"1. {curriculum_name} Core Requirements"

    for item in core_section:
        if "Course" in item:
            course = item['Course']
            processed_list.append({
                'Section': section_name,
                'Sub_Section': 'A. Mandatory Core Courses',
                'Courses in Minor': course,
                'Note': item.get('Title', 'Mandatory'),
                'Fulfilled Areas': fulfillment_map.get(course, [])
            })

        elif "group" in item:
            sub_section_name = f'B. Conditional Core: {item["group"]}'

            for course_info in item['courses']:
                course = course_info['Course']
                processed_list.append({
                    'Section': section_name,
                    'Sub_Section': sub_section_name,
                    'Courses in Minor': course,
                    'Note': course_info.get('Title', ''),
                    'Fulfilled Areas': fulfillment_map.get(course, [])
                })

    # --- ELECTIVE Courses ---
    elective_section = data.get("Electives", [])
    section_name = f"2. {curriculum_name} Elective Courses"

    for item in elective_section:
        if "Course" in item:
            course = item['Course']
            sub_section_name = "A. General Electives List"
            processed_list.append({
                'Section': section_name,
                'Sub_Section': sub_section_name,
                'Courses in Minor': course,
                'Note': item.get('Title', ''),
                'Fulfilled Areas': fulfillment_map.get(course, [])
            })

        elif "group" in item:
            sub_section_name = f'B. Conditional Elective: {item["group"]}'

            for course_info in item['courses']:
                course = course_info['Course']
                processed_list.append({
                    'Section': section_name,
                    'Sub_Section': sub_section_name,
                    'Courses in Minor': course,
                    'Note': course_info.get('Title', ''),
                    'Fulfilled Areas': fulfillment_map.get(course, [])
                })

    # --- Create DataFrame ---
    course_fulfillment_list_for_df = []
    for course_info in processed_list:
        course_dict = {
            'Section': course_info['Section'],
            'Sub_Section': course_info['Sub_Section'],
            'Courses in Minor': course_info['Courses in Minor'],
            'Note': course_info['Note']
        }

        fulfilled_areas = course_info['Fulfilled Areas']
        for gened_abbr in gened_areas:
            course_dict[gened_abbr] = 'X' if gened_abbr in fulfilled_areas else ''

        course_fulfillment_list_for_df.append(course_dict)

    course_fulfillment_df = pd.DataFrame(course_fulfillment_list_for_df).fillna('')
    course_fulfillment_df = course_fulfillment_df.set_index(['Section', 'Sub_Section', 'Courses in Minor'])
    column_order = gened_areas + ['Note']
    course_fulfillment_df = course_fulfillment_df.reindex(columns=column_order, fill_value='')

    return course_fulfillment_df

# --- 2. Styling Functions (Removed all explicit row background colors) ---

def apply_row_styles(s):
    """Applies NO explicit row background colors."""
    # Return empty string for all rows to let the environment determine the color
    return ['' for _ in s]

def apply_section_borders(styler):
    """Applies EXTREMELY prominent horizontal borders to separate major and sub-sections (Black for Major break)."""
    styles = []
    index_values = styler.index.values

    for i in range(len(index_values)):
        current_section = index_values[i][0]
        current_subsection = index_values[i][1]

        # Check for change in MAJOR section (Strong Black Border)
        if i + 1 < len(index_values) and index_values[i+1][0] != current_section:
            # Applying an EXTREMELY THICK (10px), SOLID BLACK border for Major Separation
            strong_border_rule = {
                'selector': f'tr:nth-child({i + 1}) th, tr:nth-child({i + 1}) td',
                'props': [('border-bottom', '10px solid black !important')]
            }
            styles.append(strong_border_rule)

        # Check for change in SUB-section (4px Solid Gray Border)
        elif (i + 1 < len(index_values) and
              index_values[i+1][1] != current_subsection and
              index_values[i+1][0] == index_values[i][0]):

            # Apply a THICK, SOLID GRAY border (4px) for Sub-Section Separation
            light_border_rule = {
                'selector': f'tr:nth-child({i + 1}) th, tr:nth-child({i + 1}) td',
                'props': [('border-bottom', '4px solid #cccccc !important')]
            }
            styles.append(light_border_rule)

    return styles

# --- 3. EXECUTION ---

minors_data_for_processing = minor_data[selected_minor]

# 1. Process the data
final_df = process_curriculum_json(
    selected_minor,
    minors_data_for_processing,
    gened_areas,
    course_gened_fulfillment_list
)

# 2. Apply styling and display the result
styled_df = (
    final_df.style
    # Standard properties for internal grid
    .set_properties(**{'border': '1px solid #cccccc'})
    # Apply row background styles (Now returning empty strings for all rows)
    .apply(apply_row_styles, axis=1)
    .set_caption(f'{selected_major} Major and {selected_minor} Course Fulfillment Overview')
    # Apply the dynamic, EXTREMELY THICK major section break
    .set_table_styles(apply_section_borders(final_df.style), overwrite=False)
    .set_table_styles([

        # --- BASE TABLE STYLES (Theme Neutral) ---
        {'selector': 'caption', 'props': [('font-size', '20px')]},
        # Column Headers (Top Row)
        {'selector': 'th.col_heading', 'props': [('text-align', 'center'), ('padding', '8px')]},

        # --- INDEX COLUMN SEPARATION ---
        {'selector': 'th.row_heading',
         'props': [
             ('vertical-align', 'top'),
             ('text-align', 'left'),
             ('padding', '8px'),
             ('border-bottom', '1px solid #999999 !important')
         ]},

        # Remove individual background colors for the index levels
        {'selector': 'th.row_heading.level0', 'props': []},
        {'selector': 'th.row_heading.level1', 'props': []},
        {'selector': 'th.row_heading.level2', 'props': []},

        # --- ROW HEADER HOVER EFFECT ---
        # Hover must still be visible and readable, using a light background with dark text
        {'selector': 'tr:hover th',
         'props': [('background-color', '#dddddd !important'), ('color', '#333333 !important')]},

        # When hovering over any row, highlight the data cells
        {'selector': 'tr:hover td',
         'props': [('background-color', '#eeeeee !important')]}
    ])
)

display(styled_df)